In [5]:
%reset -f

import gc
gc.collect()  # 強制 Python 回收記憶體

0

In [6]:
import os
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [7]:
date = 250402
# 設定目錄
summary_file = f"./combine/combined_{date}.csv"  # 你的總表
waveform_dir = "./waveforms"  # 波形檔案存放處
output_dir = f"./processed_data/{date}"  # 輸出目錄
output_dir_train = f"./processed_data/{date}/train"
output_dir_test = f"./processed_data/{date}/test"
os.makedirs(output_dir, exist_ok=True)
os.makedirs(output_dir_train, exist_ok=True)
os.makedirs(output_dir_test, exist_ok=True)


In [8]:
# 讀取總表
df = pd.read_csv(summary_file)

# 讀取總表
df = pd.read_csv(summary_file)

# 建立空的列表來存儲機器學習格式的數據
B_field_list = []
H_field_list = []
Duty_P_list = []
Duty_N_list = []
Turns_list = []
Hdc_list = []
frequency_list = []
temperature_list = []
volumetric_loss_list = []

# 處理每一筆測試點資料
for index, row in tqdm(df.iterrows(), total=len(df), desc="處理測試點"):
    wave_file = str(row["Wave_file_name"])  # 取得對應的波形檔案名稱

    if wave_file.lower() == "none" or wave_file.strip() == "":
        continue  # 跳過無效數據

    wave_path = os.path.join(waveform_dir, wave_file)  # 取得完整路徑

    # 檢查檔案是否存在
    if not os.path.exists(wave_path):
        print(f"檔案 {wave_file} 找不到，跳過...")
        continue

    # 讀取 B、H 波形數據
    wave_data = pd.read_csv(wave_path)

    # 假設 CSV 檔案格式固定：
    # H DATA(A/m) | B DATA(T) | I DATA(A) | V DATA(V)

    H_wave = wave_data.iloc[:, 0].values  # H DATA(A/m)
    B_wave = wave_data.iloc[:, 1].values  # B DATA(T)

    # 將 H, B 資料轉成 **橫向排列**
    H_row = list(H_wave)  # 轉成列表
    B_row = list(B_wave)  # 轉成列表

    # 存入列表
    H_field_list.append(H_row)
    B_field_list.append(B_row)

    # 存入其他數據
    frequency_list.append(row["Frequency(kHz)"])
    temperature_list.append(row["Temp"])
    volumetric_loss_list.append(row["Core_Loss(Pcv(kW/m3))"])
    Duty_P_list.append(row["Duty_P"])
    Duty_N_list.append(row["Duty_N"])
    Hdc_list.append(row["Hdc(A/m)"])
    Turns_list.append(row["N1"])


處理測試點: 100%|██████████| 1822/1822 [00:20<00:00, 89.75it/s] 


### 全部合併一個大檔案

In [9]:
# 轉換為 DataFrame，並讓 **每行代表一筆測試點**
H_field_df = pd.DataFrame(H_field_list)
B_field_df = pd.DataFrame(B_field_list)
frequency_df = pd.DataFrame(frequency_list)
temperature_df = pd.DataFrame(temperature_list)
Duty_P_df = pd.DataFrame(Duty_P_list)
Duty_N_df = pd.DataFrame(Duty_N_list)
Turns_df = pd.DataFrame(Turns_list)
Hdc_df = pd.DataFrame(Hdc_list)
volumetric_loss_df = pd.DataFrame(volumetric_loss_list)

# 儲存 CSV
H_field_df.to_csv(os.path.join(output_dir, "H_Field.csv"),
                  index=False,
                  header=False)
B_field_df.to_csv(os.path.join(output_dir, "B_Field.csv"),
                  index=False,
                  header=False)
frequency_df.to_csv(os.path.join(output_dir, "Frequency.csv"),
                    index=False,
                    header=False)
temperature_df.to_csv(os.path.join(output_dir, "Temperature.csv"),
                      index=False,
                      header=False)
Duty_P_df.to_csv(os.path.join(output_dir, "Duty_P.csv"),
                 index=False,
                 header=False)
Duty_N_df.to_csv(os.path.join(output_dir, "Duty_N.csv"),
                 index=False,
                 header=False)
Turns_df.to_csv(os.path.join(output_dir, "Turns.csv"),
                index=False,
                header=False)
Hdc_df.to_csv(os.path.join(output_dir, "Hdc.csv"), index=False, header=False)
volumetric_loss_df.to_csv(os.path.join(output_dir, "Volumetric_Loss.csv"),
                          index=False,
                          header=False)

print(f"數據拆分完成，已存入 ./processed_data/{date} 資料夾")

數據拆分完成，已存入 ./processed_data/250402 資料夾


### 分成Train 跟 Test


In [10]:
# 合併成一個大的DataFrame
full_data = pd.DataFrame({
    'H_field': H_field_list,
    'B_field': B_field_list,
    'Frequency': frequency_list,
    'Temperature': temperature_list,
    'Duty_P': Duty_P_list,
    'Duty_N': Duty_N_list,
    'Turns': Turns_list,
    'Hdc': Hdc_list,
    'Volumetric_Loss': volumetric_loss_list
})

# 隨機打亂並分割數據 (80%訓練，20%測試)
train_df, test_df = train_test_split(full_data,
                                     test_size=0.1,
                                     random_state=42,
                                     shuffle=True)

# 儲存資料
for dataset, output_dir in zip([train_df, test_df],
                               [output_dir_train, output_dir_test]):
    pd.DataFrame(dataset['H_field'].tolist()).to_csv(os.path.join(
        output_dir, "H_Field.csv"),
                                                     index=False,
                                                     header=False)
    pd.DataFrame(dataset['B_field'].tolist()).to_csv(os.path.join(
        output_dir, "B_Field.csv"),
                                                     index=False,
                                                     header=False)
    dataset[['Frequency']].to_csv(os.path.join(output_dir, "Frequency.csv"),
                                  index=False,
                                  header=False)
    dataset[['Temperature']].to_csv(os.path.join(output_dir,
                                                 "Temperature.csv"),
                                    index=False,
                                    header=False)
    dataset[['Duty_P']].to_csv(os.path.join(output_dir, "Duty_P.csv"),
                               index=False,
                               header=False)
    dataset[['Duty_N']].to_csv(os.path.join(output_dir, "Duty_N.csv"),
                               index=False,
                               header=False)
    dataset[['Turns']].to_csv(os.path.join(output_dir, "Turns.csv"),
                              index=False,
                              header=False)
    dataset[['Hdc']].to_csv(os.path.join(output_dir, "Hdc.csv"),
                            index=False,
                            header=False)
    dataset[['Volumetric_Loss']].to_csv(os.path.join(output_dir,
                                                     "Volumetric_Loss.csv"),
                                        index=False,
                                        header=False)

print(
    f"數據拆分完成，已存入 ./processed_data/{date}/train 與 ./processed_data/{date}/test 資料夾"
)


數據拆分完成，已存入 ./processed_data/250402/train 與 ./processed_data/250402/test 資料夾
